In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
!pip install datasets

In [3]:
!pip show pyarrow
!pip install --upgrade pyarrow


Name: pyarrow
Version: 15.0.0
Summary: Python library for Apache Arrow
Home-page: https://arrow.apache.org/
Author: 
Author-email: 
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy
Required-by: datasets, db-dtypes, ibis-framework, pandas-gbq


In [4]:
from torch.cuda.amp import GradScaler, autocast
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers import normalizers

In [5]:
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import random
import gc

In [6]:
%env TOKENIZERS_PARALLELISM = false

env: TOKENIZERS_PARALLELISM=false


In [7]:
dataset1 = load_dataset("opus100", "en-hi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/534319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
dataset2=load_dataset("opus100","en-mr")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/27007 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
dataset3=load_dataset("opus100","en-pa")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/107296 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
dataset4=load_dataset("opus100","en-te")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/64352 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
dataset5=load_dataset("opus100","en-ka")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/377306 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
dataset6=load_dataset("opus100","en-ml")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/822746 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
dataset7=load_dataset("opus100","en-or")

Generating test split:   0%|          | 0/1318 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/14273 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1317 [00:00<?, ? examples/s]

In [14]:
dataset8=load_dataset("opus100","en-gu")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/318306 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [15]:
dataset9=load_dataset("opus100","en-ur")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/753913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
dataset10=load_dataset("opus100","en-ta")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/227014 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
dataset11=load_dataset("opus100","bn-en")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [18]:
dataset12=load_dataset("opus100","as-en")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/138479 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [19]:
df1_train = pd.DataFrame(dataset1['train']['translation'],columns=['en','hi'])
df1_train.rename(columns={'en':'lang1','hi':'lang2'},inplace=True)
df1_val = pd.DataFrame(dataset1['validation']['translation'],columns=['en','hi'])
df1_val.rename(columns={'en':'lang1','hi':'lang2'},inplace=True)
df1_test = pd.DataFrame(dataset1['test']['translation'],columns=['en','hi'])
df1_test = pd.DataFrame(dataset1['test']['translation'],columns=['en','hi'])
df1_test.rename(columns={'en':'lang1','hi':'lang2'},inplace=True)
df1_train['lang2_id'] = 'hi'
df1_val['lang2_id'] = 'hi'
df1_test['lang2_id'] = 'hi'


In [20]:
df2_train = pd.DataFrame(dataset2['train']['translation'],columns=['en','mr'])
df2_train.rename(columns={'en':'lang1','mr':'lang2'},inplace=True)
df2_val = pd.DataFrame(dataset2['validation']['translation'],columns=['en','mr'])
df2_val.rename(columns={'en':'lang1','mr':'lang2'},inplace=True)
df2_test = pd.DataFrame(dataset2['test']['translation'],columns=['en','mr'])
df2_test.rename(columns={'en':'lang1','mr':'lang2'},inplace=True)
df2_train['lang2_id'] = 'mr'
df2_val['lang2_id'] = 'mr'
df2_test['lang2_id'] = 'mr'

In [21]:
df3_train = pd.DataFrame(dataset3['train']['translation'],columns=['en','pa'])
df3_train.rename(columns={'en':'lang1','pa':'lang2'},inplace=True)
df3_val = pd.DataFrame(dataset3['validation']['translation'],columns=['en','pa'])
df3_val.rename(columns={'en':'lang1','pa':'lang2'},inplace=True)
df3_test = pd.DataFrame(dataset3['test']['translation'],columns=['en','pa'])
df3_test.rename(columns={'en':'lang1','pa':'lang2'},inplace=True)
df3_train['lang2_id'] = 'pa'
df3_val['lang2_id'] = 'pa'
df3_test['lang2_id'] = 'pa'

In [22]:
df4_train = pd.DataFrame(dataset4['train']['translation'],columns=['en','te'])
df4_train.rename(columns={'en':'lang1','te':'lang2'},inplace=True)
df4_val = pd.DataFrame(dataset4['validation']['translation'],columns=['en','te'])
df4_val.rename(columns={'en':'lang1','te':'lang2'},inplace=True)
df4_test = pd.DataFrame(dataset4['test']['translation'],columns=['en','te'])
df4_test.rename(columns={'en':'lang1','te':'lang2'},inplace=True)
df4_train['lang2_id'] = 'te'
df4_val['lang2_id'] = 'te'
df4_test['lang2_id'] = 'te'

In [23]:
df5_train = pd.DataFrame(dataset5['train']['translation'],columns=['en','ka'])
df5_train.rename(columns={'en':'lang1','ka':'lang2'},inplace=True)
df5_val = pd.DataFrame(dataset5['validation']['translation'],columns=['en','ka'])
df5_val.rename(columns={'en':'lang1','ka':'lang2'},inplace=True)
df5_test = pd.DataFrame(dataset5['test']['translation'],columns=['en','ka'])
df5_test.rename(columns={'en':'lang1','ka':'lang2'},inplace=True)
df5_train['lang2_id'] = 'ka'
df5_val['lang2_id'] = 'ka'
df5_test['lang2_id'] = 'ka'

In [24]:
df6_train = pd.DataFrame(dataset6['train']['translation'],columns=['en','ml'])
df6_train.rename(columns={'en':'lang1','ml':'lang2'},inplace=True)
df6_val = pd.DataFrame(dataset6['validation']['translation'],columns=['en','ml'])
df6_val.rename(columns={'en':'lang1','ml':'lang2'},inplace=True)
df6_test = pd.DataFrame(dataset6['test']['translation'],columns=['en','ml'])
df6_test.rename(columns={'en':'lang1','ml':'lang2'},inplace=True)
df6_train['lang2_id'] = 'ml'
df6_val['lang2_id'] = 'ml'
df6_test['lang2_id'] = 'ml'

In [25]:
df7_train = pd.DataFrame(dataset7['train']['translation'],columns=['en','or'])
df7_train.rename(columns={'en':'lang1','or':'lang2'},inplace=True)
df7_val = pd.DataFrame(dataset7['validation']['translation'],columns=['en','or'])
df7_val.rename(columns={'en':'lang1','or':'lang2'},inplace=True)
df7_test = pd.DataFrame(dataset7['test']['translation'],columns=['en','or'])
df7_test.rename(columns={'en':'lang1','or':'lang2'},inplace=True)
df7_train['lang2_id'] = 'or'
df7_val['lang2_id'] = 'or'
df7_test['lang2_id'] = 'or'

In [26]:
df8_train = pd.DataFrame(dataset8['train']['translation'],columns=['en','gu'])
df8_train.rename(columns={'en':'lang1','gu':'lang2'},inplace=True)
df8_val = pd.DataFrame(dataset8['validation']['translation'],columns=['en','gu'])
df8_val.rename(columns={'en':'lang1','gu':'lang2'},inplace=True)
df8_test = pd.DataFrame(dataset8['test']['translation'],columns=['en','gu'])
df8_test.rename(columns={'en':'lang1','gu':'lang2'},inplace=True)
df8_train['lang2_id'] = 'gu'
df8_val['lang2_id'] = 'gu'
df8_test['lang2_id'] = 'gu'

In [27]:
df9_train = pd.DataFrame(dataset9['train']['translation'],columns=['en','ur'])
df9_train.rename(columns={'en':'lang1','ur':'lang2'},inplace=True)
df9_val = pd.DataFrame(dataset9['validation']['translation'],columns=['en','ur'])
df9_val.rename(columns={'en':'lang1','ur':'lang2'},inplace=True)
df9_test = pd.DataFrame(dataset9['test']['translation'],columns=['en','ur'])
df9_test.rename(columns={'en':'lang1','ur':'lang2'},inplace=True)
df9_train['lang2_id'] = 'ur'
df9_val['lang2_id'] = 'ur'
df9_test['lang2_id'] = 'ur'

In [28]:
df10_train = pd.DataFrame(dataset10['train']['translation'],columns=['en','ta'])
df10_train.rename(columns={'en':'lang1','ta':'lang2'},inplace=True)
df10_val = pd.DataFrame(dataset10['validation']['translation'],columns=['en','ta'])
df10_val.rename(columns={'en':'lang1','ta':'lang2'},inplace=True)
df10_test = pd.DataFrame(dataset10['test']['translation'],columns=['en','ta'])
df10_test.rename(columns={'en':'lang1','ta':'lang2'},inplace=True)
df10_train['lang2_id'] = 'ta'
df10_val['lang2_id'] = 'ta'
df10_test['lang2_id'] = 'ta'

In [29]:
df11_train = pd.DataFrame(dataset11['train']['translation'],columns=['bn','en'])
df11_train.rename(columns={'en':'lang1','bn':'lang2'},inplace=True)
df11_val = pd.DataFrame(dataset11['validation']['translation'],columns=['en','bn'])
df11_val.rename(columns={'en':'lang1','bn':'lang2'},inplace=True)
df11_test = pd.DataFrame(dataset11['test']['translation'],columns=['en','bn'])
df11_test.rename(columns={'en':'lang1','bn':'lang2'},inplace=True)
df11_train['lang2_id'] = 'bn'
df11_val['lang2_id'] = 'bn'
df11_test['lang2_id'] = 'bn'

In [30]:
df12_train = pd.DataFrame(dataset12['train']['translation'],columns=['as','en'])
df12_train.rename(columns={'en':'lang1','as':'lang2'},inplace=True)
df12_val = pd.DataFrame(dataset12['validation']['translation'],columns=['as','en'])
df12_val.rename(columns={'en':'lang1','as':'lang2'},inplace=True)
df12_test = pd.DataFrame(dataset12['test']['translation'],columns=['en','as'])
df12_test.rename(columns={'en':'lang1','as':'lang2'},inplace=True)
df12_train['lang2_id'] = 'as'
df12_val['lang2_id'] = 'as'
df12_test['lang2_id'] = 'as'

In [31]:
train_df = pd.concat([df1_train,df2_train,df3_train,df4_train,df5_train,df6_train,df7_train,df8_train,df9_train,df10_train,df11_train,df12_train]).reset_index(drop=True)
val_df = pd.concat([df1_val,df2_val,df3_val,df4_val,df5_val,df6_val,df7_val,df8_val,df9_val,df10_val,df11_val,df12_val]).reset_index(drop=True)
test_df = pd.concat([df1_test,df2_test,df3_test,df4_test,df5_test,df6_test,df7_test,df8_test,df9_test,df10_test,df11_test,df12_test]).reset_index(drop=True)

In [32]:
train_df

,lang1,lang2,lang2_id
0,"Other, Private Use","अन्य, निज़ी उपयोग",hi
1,[SCREAMING],ऊबड़ .,hi
2,Spouse,जीवनसाथी,hi
3,I will never salute you!,- तुम एक कमांडर कभी नहीं होगा!,hi
4,and the stars and the trees bow themselves;,और तारे और वृक्ष सजदा करते है;,hi
...,...,...,...
4385006,The item could not be deleted due to an error,একটি ত্ৰুটিৰ দৰুন এই বস্তুটি আঁতৰুৱা নাযায়,as
4385007,New,নতুন,as
4385008,Address formatting,মুক্ত/ব্যস্ত তথ্য,as
4385009,Move selected headers to top,নিৰ্বাচিত হেডাৰসমূহ ওপৰলৈ স্থানান্তৰ কৰক,as


In [33]:
len(train_df), len(val_df), len(test_df)

(4385011, 23317, 23318)

In [34]:
full_df = pd.concat([train_df,val_df])
lang1,lang2 = list(full_df['lang1']), list(full_df['lang2'])
full = lang1+lang2
random.shuffle(full)


In [35]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 18 11:29:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [36]:

bert_tokenizer = Tokenizer(WordPiece(unk_token="<unk>"))
bert_tokenizer.normalizer = normalizers.Sequence([Lowercase()])
bert_tokenizer.pre_tokenizer = Whitespace()
bert_tokenizer.decoder = decoders.WordPiece()
trainer = WordPieceTrainer(special_tokens=["<unk>","<pad>","<s-en>","<s-hi>","<s-mr>","<s-pa>","<s-te>","<s-ka>","<s-ml>","<s-or>","<s-gu>","<s-ur>","<s-ta>","<s-bn>","<s-as>","</s>"])
bert_tokenizer.train_from_iterator(full,trainer)
bert_tokenizer.enable_padding(
    pad_id=bert_tokenizer.token_to_id('<pad>'),
    length=128,
    pad_token='<pad>'
)

In [37]:
bert_tokenizer.enable_truncation(128)

base = Path('translator/tokenizer',)
base.mkdir(exist_ok=True,parents=True)
bert_tokenizer.save(str(base / 'en_hi_mr_pa_te_ka_ml_or_gu_ur_ta_bn_as.json'))
x = bert_tokenizer.encode(f"<s-hi>{lang2[12345]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-mr>{lang2[555000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids), '\n\n')
x = bert_tokenizer.encode(f"<s-pa>{lang2[600000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-te>{lang2[700000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')
x = bert_tokenizer.encode(f"<s-ka>{lang2[1000000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-ml>{lang2[1500000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-or>{lang2[1965000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-gu>{lang2[2100000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')
x = bert_tokenizer.encode(f"<s-ur>{lang2[2900000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-ta>{lang2[3200000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')
x = bert_tokenizer.encode(f"<s-bn>{lang2[4000000]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids),'\n\n')

x = bert_tokenizer.encode(f"<s-as>{lang2[-111]}</s>")
for a,b in zip(x.ids, x.tokens):
    if b!= '<pad>':
        print(f'{a} : {b}')

print('\n',bert_tokenizer.decode(x.ids))
print('en',bert_tokenizer.token_to_id('<s-en>'))
print('hi',bert_tokenizer.token_to_id('<s-hi>'))
print('mr',bert_tokenizer.token_to_id('<s-mr>'))
print('pa',bert_tokenizer.token_to_id('<s-pa>'))
print('te',bert_tokenizer.token_to_id('<s-te>'))
print('ka',bert_tokenizer.token_to_id('<s-ka>'))
print('ml',bert_tokenizer.token_to_id('<s-ml>'))
print('or',bert_tokenizer.token_to_id('<s-or>'))
print('gu',bert_tokenizer.token_to_id('<s-gu>'))
print('ur',bert_tokenizer.token_to_id('<s-ur>'))
print('ta',bert_tokenizer.token_to_id('<s-ta>'))
print('bn',bert_tokenizer.token_to_id('<s-bn>'))
print('as',bert_tokenizer.token_to_id('<s-as>'))
print('eos',bert_tokenizer.token_to_id('</s>'))
print('pad',bert_tokenizer.token_to_id('<pad>'))
class Dataset:
    def __init__(self,df):
        self.df = df
    def __len__(self,):
        return len(self.df)
    def __getitem__(self,idx):
        sample = self.df.iloc[idx,:]
        en,lang2 = sample['lang1'], sample['lang2']
        if sample['lang2_id']=='hi':
          start_token = "<s-hi>"
        elif sample['lang2_id']=='mr':
          start_token = "<s-mr>"
        elif sample['lang2_id']=='pa':
          start_token = "<s-pa>"
        elif sample['lang2_id']=='te':
          start_token = "<s-te>"
        elif sample['lang2_id']=='ka':
          start_token = "<s-ka>"
        elif sample['lang2_id']=='ml':
          start_token = "<s-ml>"
        elif sample['lang2_id']=='or':
          start_token = "<s-or>"
        elif sample['lang2_id']=='gu':
          start_token = "<s-gu>"
        elif sample['lang2_id']=='ur':
          start_token = "<s-ur>"
        elif sample['lang2_id']=='ta':
          start_token = "<s-ta>"
        elif sample['lang2_id']=='bn':
          start_token = "<s-bn>"
        elif sample['lang2_id']=='as':
          start_token = "<s-as>"
        else:
          start_token = "<s-en>"
        en = bert_tokenizer.encode(f'<s-en>{en.strip()}</s>').ids
        l2 = bert_tokenizer.encode(f'{start_token}{lang2.strip()}</s>').ids
        l2_shift = l2.copy()
        l2_shift[:-1] = l2[1:]
        l2_shift[-1] = bert_tokenizer.token_to_id('<pad>')

        en = torch.tensor(en,dtype=torch.long)
        l2 = torch.tensor(l2,dtype=torch.long)
        l2_shift = torch.tensor(l2_shift,dtype=torch.long)
        l2_shift[l2_shift==1]=-100
        return en,l2,l2_shift


3 : <s-hi>
26659 : आपने
24550 : चैन
2512 : ##ल
19924 : मालिक
4682 : का
28256 : विशेष
2244 : ##ा
25260 : ##धिक
4330 : ##ार
20 : %
32 : 1
4499 : से
15367 : वापस
5893 : ले
8763 : लिया
4255 : है
29 : .
15 : </s>

 आपने चैनल मालिक का विशेषाधिकार % 1 से वापस ले लिया है. 


4 : <s-mr>
17332 : त्या
488 : आ
18994 : ##कर्
2694 : ##ष
2245 : ##क
29839 : आहेत
29 : .
15 : </s>

 त्या आकर्षक आहेत. 


5 : <s-pa>
21 : &
9845 : info
10249 : ##cent
4113 : ##er
42 : ;
12025 : ਦੀ
24877 : ਵਰਤੋਂ
15 : </s>

 & infocenter ; ਦੀ ਵਰਤੋਂ 


6 : <s-te>
940 : ఈ
11397 : ఒక
967 : న
27 : ,
952 : గ
2410 : ##ే
16167 : ##మ్
950 : క
2948 : ##ఠ
15150 : ##ిన
22545 : ##మైన
28661 : వచ్చింది
29 : .
15 : </s>

 ఈ ఒక న, గేమ్ కఠినమైన వచ్చింది. 


7 : <s-ka>
6356 : პა
20039 : ##რო
18334 : ##ლის
1156 : დ
16954 : ##ია
20060 : ##ლო
2266 : ##გ
14868 : ##ში
1153 : ა
21825 : ##სა
21825 : ##სა
2744 : ##ხ
2364 : ##ი
28892 : გამო
21825 : ##სა
2744 : ##ხ
24562 : ##ულ
11509 : ##ება
15 : </s>

 პაროლის დიალოგში ასასახი გამოსახულება 


8 : <s-

In [38]:
train_ds = Dataset(train_df)
val_ds = Dataset(val_df)

In [39]:
print('english tokens\n',train_ds[0][0])
print('lang2 tokens\n',train_ds[0][1])
print('right-shifted lang2 tokens\n',train_ds[0][2])


english tokens
 tensor([    2,  4950,    27, 10973,  5774,    15,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1

In [40]:
class RMSNorm(nn.Module):
    def __init__(self, d, p=-1., eps=1e-8, bias=False):
        """
            Root Mean Square Layer Normalization
        :param d: model size
        :param p: partial RMSNorm, valid value [0, 1], default -1.0 (disabled)
        :param eps:  epsilon value, default 1e-8
        :param bias: whether use bias term for RMSNorm, disabled by
            default because RMSNorm doesn't enforce re-centering invariance.
        """
        super(RMSNorm, self).__init__()

        self.eps = eps
        self.d = d
        self.p = p
        self.bias = bias

        self.scale = nn.Parameter(torch.ones(d))
        self.register_parameter("scale", self.scale)

        if self.bias:
            self.offset = nn.Parameter(torch.zeros(d))
            self.register_parameter("offset", self.offset)

    def forward(self, x):
        if self.p < 0. or self.p > 1.:
            norm_x = x.norm(2, dim=-1, keepdim=True)
            d_x = self.d
        else:
            partial_size = int(self.d * self.p)
            partial_x, _ = torch.split(x, [partial_size, self.d - partial_size], dim=-1)

            norm_x = partial_x.norm(2, dim=-1, keepdim=True)
            d_x = partial_size

        rms_x = norm_x * d_x ** (-1. / 2)
        x_normed = x / (rms_x + self.eps)

        if self.bias:
            return self.scale * x_normed + self.offset

        return self.scale * x_normed


class MultiheadAttention(nn.Module):
    def __init__(self, dim, n_heads, dropout=0.):
        super().__init__()
        self.dim = dim
        self.n_heads = n_heads
        assert dim % n_heads == 0, 'dim should be div by n_heads'
        self.head_dim = self.dim // self.n_heads
        self.q = nn.Linear(dim,dim,bias=False)
        self.k = nn.Linear(dim,dim,bias=False)
        self.v = nn.Linear(dim,dim,bias=False)
        self.attn_dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5
        self.out_proj = nn.Linear(dim,dim,bias=False)

    def forward(self,q,k,v,mask=None):
        batch,t,c = q.shape
        q = self.q(q)
        k = self.k(k)
        v = self.v(v)
        q = q.view(batch,q.size(1),self.n_heads,self.head_dim).permute(0,2,1,3)
        k = k.view(batch,k.size(1),self.n_heads,self.head_dim).permute(0,2,1,3)
        v = v.view(batch,v.size(1),self.n_heads,self.head_dim).permute(0,2,1,3)

        qkT = torch.matmul(q,k.transpose(-1,-2)) * self.scale
        qkT = self.attn_dropout(qkT)

        if mask is not None:
            mask = mask.to(dtype=qkT.dtype,device=qkT.device)
            a,b = qkT.size(-2), qkT.size(-1)
            qkT = qkT.masked_fill(mask[:,:,:a,:b]==0,float('-inf'))

        qkT = F.softmax(qkT,dim=-1)

        attn = torch.matmul(qkT,v)
        attn = attn.permute(0,2,1,3).contiguous().view(batch,t,c)
        out = self.out_proj(attn)
        return out



class FeedForward(nn.Module):
    def __init__(self,dim,dropout=0.):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim,dim*4,bias=False),
            nn.Dropout(dropout),
            nn.GELU(),
            nn.Linear(dim*4,dim,bias=False)
        )

    def forward(self, x):
        return self.feed_forward(x)



class EncoderBlock(nn.Module):
    def __init__(self, dim, n_heads, attn_dropout=0., mlp_dropout=0.):
        super().__init__()
        self.attn = MultiheadAttention(dim,n_heads,attn_dropout)
        self.ffd = FeedForward(dim,mlp_dropout)
        self.ln_1 = RMSNorm(dim)
        self.ln_2 = RMSNorm(dim)

    def forward(self,x,mask=None):
        x = self.ln_1(x)
        x = x + self.attn(x,x,x,mask)
        x = self.ln_2(x)
        x = x + self.ffd(x)
        return x



class DecoderBlock(nn.Module):
    def __init__(self, dim, n_heads, attn_dropout=0., mlp_dropout=0.):
        super().__init__()
        self.self_attn = MultiheadAttention(dim,n_heads,attn_dropout)
        self.cross_attn = MultiheadAttention(dim,n_heads,attn_dropout)
        self.ln_1 = RMSNorm(dim)
        self.ln_2 = RMSNorm(dim)
        self.ln_3 = RMSNorm(dim)
        self.ffd = FeedForward(dim,mlp_dropout)

    def forward(self, x, enc_out, src_mask, tgt_mask):
        x = self.ln_1(x)
        x = x + self.self_attn(x,x,x,tgt_mask)
        x = self.ln_2(x)
        x = x + self.cross_attn(x,enc_out,enc_out,src_mask) # decoder: q, encoder: k,v
        x = self.ln_3(x)
        x = x + self.ffd(x)

        return x



class Embedding(nn.Module):
    def __init__(self,vocab_size,max_len,dim):
        super().__init__()
        self.max_len = max_len
        self.class_embedding = nn.Embedding(vocab_size,dim)
        self.pos_embedding = nn.Embedding(max_len,dim)
    def forward(self,x):
        x = self.class_embedding(x)
        pos = torch.arange(0,x.size(1),device=x.device)
        x = x + self.pos_embedding(pos)
        return x



class Seq2SeqTransformer(nn.Module):
    def __init__(self, config):

        super().__init__()

        self.embedding = Embedding(config['vocab_size'],config['max_len'],config['dim'])

        self.depth = config['depth']
        self.encoders = nn.ModuleList([
            EncoderBlock(
                dim=config['dim'],
                n_heads=config['n_heads'],
                attn_dropout=config['attn_dropout'],
                mlp_dropout=config['mlp_dropout']
            ) for _ in range(self.depth)
        ])
        self.decoders = nn.ModuleList([
            DecoderBlock(
                dim=config['dim'],
                n_heads=config['n_heads'],
                attn_dropout=config['attn_dropout'],
                mlp_dropout=config['mlp_dropout']
            ) for _ in range(self.depth)
        ])

        self.ln_f = RMSNorm(config['dim'])
        self.lm_head = nn.Linear(config['dim'],config['vocab_size'],bias=False)

        self.embedding.class_embedding.weight = self.lm_head.weight

        self.pad_token_id = config['pad_token_id']
        self.register_buffer('tgt_mask',torch.tril(torch.ones(1,1,config['max_len'],config['max_len'])))

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def create_src_mask(self,src):
        return (src != self.pad_token_id).unsqueeze(1).unsqueeze(2) # N, 1, 1, src_len

    def forward(self, src, tgt, labels=None):

        src_mask = self.create_src_mask(src)

        enc_out = self.embedding(src)
        dec_out = self.embedding(tgt)

        for i in range(self.depth):
            enc_out = self.encoders[i](enc_out,mask=src_mask)
            dec_out = self.decoders[i](dec_out,enc_out,src_mask=src_mask,tgt_mask=self.tgt_mask)

        dec_out = self.ln_f(dec_out)

        if labels is not None:
            lm_logits = self.lm_head(dec_out)
            loss = F.cross_entropy(lm_logits.view(-1, lm_logits.shape[-1]), labels.view(-1))
            return loss

        lm_logits = self.lm_head(dec_out[:,[-1],:])
        return lm_logits

    def generate(self,src,max_tokens=80,temperature=1.0,deterministic=False,eos=5,bos=None):
        tgt = torch.ones(1,1).long() * bos
        tgt = tgt.to(src.device)
        for _ in range(max_tokens):
            out = self(src,tgt)
            out = out[:,-1,:] / temperature
            probs = F.softmax(out,dim=-1)
            if deterministic:
                next_token = torch.argmax(probs,dim=-1,keepdim=True)
            else:
                next_token = torch.multinomial(probs,num_samples=1)
            tgt = torch.cat([tgt,next_token],dim=1)
            if next_token.item() == eos:
                break

        return tgt.cpu().flatten()

In [41]:
config = {
    'dim': 128,
    'n_heads': 4,
    'attn_dropout': 0.1,
    'mlp_dropout': 0.1,
    'depth': 8,
    'vocab_size': bert_tokenizer.get_vocab_size(),
    'max_len': 128,
    'pad_token_id': bert_tokenizer.token_to_id('<pad>')
}
config

{'dim': 128,
 'n_heads': 4,
 'attn_dropout': 0.1,
 'mlp_dropout': 0.1,
 'depth': 8,
 'vocab_size': 30000,
 'max_len': 128,
 'pad_token_id': 1}

In [42]:
model = Seq2SeqTransformer(config).to('cuda')
print(sum([p.numel() for p in model.parameters() if p.requires_grad]))

7531648


In [43]:
print(model)

Seq2SeqTransformer(
  (embedding): Embedding(
    (class_embedding): Embedding(30000, 128)
    (pos_embedding): Embedding(128, 128)
  )
  (encoders): ModuleList(
    (0-7): 8 x EncoderBlock(
      (attn): MultiheadAttention(
        (q): Linear(in_features=128, out_features=128, bias=False)
        (k): Linear(in_features=128, out_features=128, bias=False)
        (v): Linear(in_features=128, out_features=128, bias=False)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (out_proj): Linear(in_features=128, out_features=128, bias=False)
      )
      (ffd): FeedForward(
        (feed_forward): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=False)
          (1): Dropout(p=0.1, inplace=False)
          (2): GELU(approximate='none')
          (3): Linear(in_features=512, out_features=128, bias=False)
        )
      )
      (ln_1): RMSNorm()
      (ln_2): RMSNorm()
    )
  )
  (decoders): ModuleList(
    (0-7): 8 x DecoderBlock(
      (self_attn): Mul

In [44]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128,shuffle=True,pin_memory=True,num_workers=2)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=128,shuffle=False,pin_memory=True,num_workers=2)
print(len(train_dl), len(val_dl))

34258 183


In [45]:
test_samples = [(test_df.loc[i,'lang1'],test_df.loc[i,'lang2'],test_df.loc[i,'lang2_id']) for i in range(len(test_df))]

In [46]:
epochs = 5
train_losses = []
valid_losses = []
best_val_loss = 1e9

all_tl = []
all_lr = []

optim = torch.optim.Adam(model.parameters(),lr=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optim,
    T_0=250,
    eta_min=1e-8
)

scaler = GradScaler()

In [ ]:
for ep in tqdm(range(epochs)):
    model.train()
    trl = 0.
    tprog = tqdm(enumerate(train_dl),total=len(train_dl))
    for i, batch in tprog:
        with autocast():
            src, tgt, labels = [b.to('cuda') for b in batch]
            loss = model(src,tgt,labels)
            scaler.scale(loss).backward()
            scaler.unscale_(optim)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0, norm_type=2)
            scaler.step(optim)
            scaler.update()
            optim.zero_grad()
            sched.step(ep + i / len(train_dl))
            all_lr.append(sched.get_last_lr())
            trl += loss.item()
            all_tl.append(loss.item())
            tprog.set_description(f'train step loss: {loss.item():.4f}')
    train_losses.append(trl/len(train_dl))

    gc.collect()
    torch.cuda.empty_cache()

    model.eval()
    with torch.no_grad():
        vrl = 0.
        vprog = tqdm(enumerate(val_dl),total=len(val_dl))
        for i, batch in vprog:
            with autocast():
                src, tgt, labels = [b.to('cuda') for b in batch]
                loss = model(src,tgt,labels)
                vrl += loss.item()
                vprog.set_description(f'valid step loss: {loss.item():.4f}')
        vloss = vrl/len(val_dl)
        valid_losses.append(vloss)
        print(f'epoch {ep} | train_loss: {train_losses[-1]:.4f} valid_loss: {valid_losses[-1]:.4f}')

        if vloss < best_val_loss:
            best_val_loss = vloss

            print('saving best model...')
            sd = model.state_dict()
            torch.save(sd,'./translator/best_model.pt')

        print('saving epoch checkpoint...')
        sd = model.state_dict()
        torch.save(sd,f'./translator/checkpoint_model_epoch{ep}.pt')

        print('predicting with current epoch model...')
        for (src,tgt,lang_id) in random.choices(test_samples,k=5):
            input_ids = bert_tokenizer.encode(f"<s-en>{src}</s>").ids
            input_ids = torch.tensor(input_ids,dtype=torch.long).unsqueeze(0).to('cuda')
            if lang_id == 'hi':
                bos = bert_tokenizer.token_to_id('<s-hi>')
            elif lang_id =='mr':
                bos = bert_tokenizer.token_to_id('<s-mr>')
            elif lang_id =='pa':
                bos = bert_tokenizer.token_to_id('<s-pa>')
            elif lang_id =='te':
                bos = bert_tokenizer.token_to_id('<s-te>')
            elif lang_id =='ka':
                bos = bert_tokenizer.token_to_id('<s-ka>')
            elif lang_id =='ml':
                bos = bert_tokenizer.token_to_id('<s-ml>')
            elif lang_id =='or':
                bos = bert_tokenizer.token_to_id('<s-or>')
            elif lang_id =='gu':
                bos = bert_tokenizer.token_to_id('<s-gu>')
            elif lang_id =='ur':
                bos = bert_tokenizer.token_to_id('<s-ur>')
            elif lang_id =='ta':
                bos = bert_tokenizer.token_to_id('<s-ta>')
            elif lang_id =='bn':
                bos = bert_tokenizer.token_to_id('<s-bn>')
            elif lang_id =='as':
                bos = bert_tokenizer.token_to_id('<s-as>')
            else :
                bos = bert_tokenizer.token_to_id('<s-en>')
            tgt_out = model.generate(input_ids,bos=bos,deterministic=True)
            tgt_out = bert_tokenizer.decode(tgt_out.numpy())
            print(f'\nEN: {src} \n{lang_id.upper()}: {tgt} \n{lang_id.upper()}_MODEL: {tgt_out}\n')

        print('-'*30,'\n\n')

    gc.collect()
    torch.cuda.empty_cache()



  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/34258 [00:00<?, ?it/s]

  0%|          | 0/183 [00:00<?, ?it/s]

epoch 0 | train_loss: 4.1664 valid_loss: 3.4616
saving best model...
saving epoch checkpoint...
predicting with current epoch model...

EN: Public unlisted 
AS: ৰাজহুৱাভাৱে তালিকাৰ বাহিৰ 
AS_MODEL: প্ৰদানাৰণিতভাৱে


EN: Display the next thread 
GU: આગળનો થ્રેડ દર્શાવો 
GU_MODEL: આગળના થ્રેડો દર્શાવો


EN: I- - Oh- - It's too big around. 
TE: ఇది చుట్టూ చాలా పెద్దది. 
TE_MODEL: నేను - ఓహ్ - ఓహ్, ఇది చాలా చాలా చాలా.


EN: Kemal Taher from Erbil remarked, “I don’t know what to say about this lioness of Kurdistan, wishing them success on the battlefield, May God protect you all”. 
OR: ଏରବିଲରୁ କେମଲ ତାହେର ମତ ଦେଇଛନ୍ତି, "କୁରଦିସ୍ଥାନର ଏହି ସିଂହୀଙ୍କ ବାବଦରେ ମୁଁ କଣ କହିବି ଜାଣେନା, ତାଙ୍କୁ ଯୁଦ୍ଧଭୂମିରେ ସଫଳତା କାମନା କରୁଛି, ଭଗବାନ ଆପଣ ସଭିଙ୍କୁ ରକ୍ଷାକରନ୍ତୁ" । 
OR_MODEL: କେଟେରାମାମାମାମାମାମାମାମାମାମାମାମାମାମାମେରୁ ' କିମେମେ ' କିମେମାମାମା କିମେମାମା କିମାମାମା କିମାମେରଣ


EN: No bookmarks 
GU: બુકમાર્કો નથી 
GU_MODEL: બુકમાર્ક નથી

------------------------------ 




  0%|          | 0/34258 [00:00<?, ?it/s]

  0%|          | 0/183 [00:00<?, ?it/s]

epoch 1 | train_loss: 2.9415 valid_loss: 3.0779
saving best model...
saving epoch checkpoint...
predicting with current epoch model...

EN: That's not even warm. 
TE: ఆ వేడి లేదు. ఇది వేడిగా ఉంది. 
TE_MODEL: ఆ అంత లేదు.


EN: Sudan 
HI: सूडान 
HI_MODEL: आसपान


EN: Your emails 
TA: tamilpc@ ambalam. com 
TA_MODEL: உங்கள் மின்னஞ்சல்


EN: Processor wakeups per second: 
GU: પ્રોસેસર સેકન્ડ પ્રતિ જાગે છે: 
GU_MODEL: પ્રક્રિયા સ્પર્શup માંગે વધારે :


EN: And were you to wonder, then wondrous indeed is the saying of those who say: "What! After we have been reduced to dust, shall we be created afresh?" They are the ones who disbelieved in their Lord; they are the ones who shall have shackles around their necks. They shall be the inmates of the Fire, wherein they will abide for ever. 
UR: اور اگر تمہیں کسی بات پر تعّجب ہے تو تّعجب کی بات ان لوگوں کا یہ قول ہے کہ کیا ہم خاک ہوجانے کے بعد بھی نئے سرے سے دوبارہ پیدا کئے جائیں گے .یہی وہ لوگ ہیں جنہوں نے اپنے پروردگار کا انکار کیا ہے اور ان ہی ک

  0%|          | 0/34258 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_losses,color='red',label='train loss')
plt.plot(valid_losses,color='orange',label='validation loss')
plt.legend()
plt.show()


In [ ]:
plt.plot(all_tl,color='red')
plt.title('global train step loss')
plt.show()

In [ ]:
plt.plot(all_lr,color='blue')
plt.title('global LR scheduling')
plt.show()

In [ ]:
sd = torch.load('./translator/best_model.pt')
model.load_state_dict(sd)
model.eval()

In [ ]:
with torch.no_grad():
    for i, (src,tgt,lang_id) in enumerate(random.choices(test_samples,k=250)):
        input_ids = bert_tokenizer.encode(f"<s-en>{src}</s>").ids
        input_ids = torch.tensor(input_ids,dtype=torch.long).unsqueeze(0).to('cuda')
        deterministic = False
        if i > 125:
            deterministic = True
        if lang_id == 'hi':
            bos = bert_tokenizer.token_to_id('<s-hi>')
        elif lang_id == 'mr':
            bos = bert_tokenizer.token_to_id('<s-mr>')
        elif lang_id == 'pa':
            bos = bert_tokenizer.token_to_id('<s-pa>')
        elif lang_id == 'te':
            bos = bert_tokenizer.token_to_id('<s-te>')
        elif lang_id == 'ka':
            bos = bert_tokenizer.token_to_id('<s-ka>')
        elif lang_id == 'ml':
            bos = bert_tokenizer.token_to_id('<s-ml>')
        elif lang_id == 'or':
            bos = bert_tokenizer.token_to_id('<s-or>')
        elif lang_id == 'gu':
            bos = bert_tokenizer.token_to_id('<s-gu>')
        elif lang_id == 'ur':
            bos = bert_tokenizer.token_to_id('<s-ur>')
        elif lang_id == 'ta':
            bos = bert_tokenizer.token_to_id('<s-ta>')
        elif lang_id == 'bn':
            bos = bert_tokenizer.token_to_id('<s-bn>')
        elif lang_id == 'as':
            bos = bert_tokenizer.token_to_id('<s-as>')
        else:
            bos = bert_tokenizer.token_to_id('<s-en>')
        if not deterministic:
            temperature = np.random.uniform(0.8,1.2,(1,))[0]
        else:
            temperature = 1.0
        tgt_out = model.generate(input_ids,temperature=temperature,deterministic=deterministic,bos=bos)
        tgt_out = bert_tokenizer.decode(tgt_out.numpy())
        print(f'{deterministic=}, {temperature=:.3f}')
        print(f'\nEN: {src} \n{lang_id.upper()}: {tgt} \n{lang_id.upper()}_MODEL: {tgt_out}\n\n')
